# Music

## Analyze Spotify playlists tracks

In [17]:
import os
import pandas as pd
from pandasql import sqldf as sql_df

data_path = os.getcwd().replace("\\", "/")[:-6] + "data/" # 6 = len("assets")
spotify_playlists_tracks = pd.read_csv(data_path + '240406SpotifyPlaylistsTracks.tsv', sep='\t', header=0)
# spotify_playlists_tracks[:1].T
# 	added_at          	2018-12-30 01:31:34
# 	added_by          	darkjavier
# 	album_name        	The Brain Dance
# 	album_release_date	2016-09-30
# 	album_n_tracks    	1.0
# 	artists_name      	Animals As Leaders
# 	duration_ms       	420746
# 	id                	5yKbTecdbmBikj6UbZDEhX
# 	name              	The Brain Dance
# 	popularity        	0
# 	track_number      	1
# 	playlist_name     	progressive rock
spotify_playlists_tracks[["playlist_name","artists_name","name","id"]][:2].T

,0,1
playlist_name,progressive rock,progressive rock
artists_name,Animals As Leaders,Ghost Toast
name,The Brain Dance,Gordius
id,5yKbTecdbmBikj6UbZDEhX,6yREC4ZZpHAuygarwVoeXE


In [30]:
spotify_duplicates = sql_df('''
  SELECT artists_name, name as track_name, id as track_id, count(*) as n_duplicates
  FROM spotify_playlists_tracks
  GROUP BY artists_name, track_name, track_id
  HAVING n_duplicates>1
  ORDER BY n_duplicates DESC, artists_name ASC, track_name ASC
''')
spotify_duplicates

,artists_name,track_name,track_id,n_duplicates
0,65daysofstatic,Debutante,0CpaNwilKJGqn9WF6K6aRV,4
1,Ajattara,Intro,5y7YPdvzfYuyp6nvyY7Cww,4
2,Amy Macdonald,This Is The Life,3oyqGFcN7m3jk4alkNI3j9,4
3,Anan Ryoko,Refrain,0bXZImFaGuFsAyszjQ5DSG,4
4,Avril Lavigne,When You're Gone,4kxND2rbPDrdb7Sgoo1Fbh,4
...,...,...,...,...
1583,thenewno2,Interception,63mPSly0PaZwngWnf4wa3V,2
1584,zircon,The End,24HZkXA56U0OUrkJ8fgzpv,2
1585,Édith Piaf,"Non, je ne regrette rien",0D4gIeyrMtxOAvCWsTHN9x,2
1586,Šventinis bankuchenas,Vasaraš Atostogaš,7HWW5fbqCyKZZNajEaUxxG,2


In [26]:
songs_per_playlist = sql_df("""
SELECT playlist_name, count(*) as n_tracks
FROM spotify_playlists_tracks
GROUP BY playlist_name
ORDER BY playlist_name ASC
""") # LIMIT 6
songs_per_playlist.to_csv("../data/songs_per_playlist.csv")
songs_per_playlist

,playlist_name,n_tracks
0,LotR chronologically,37
1,Menuo Juodaragis 2022,147
2,acoust&ballads&sg-sw 10s and beyond,165
3,acoust&ballads&sg-sw/00s,226
4,aggrotech & harsh electro,45
...,...,...
67,tribal,85
68,trip-hop,77
69,uplifting ost,307
70,woods and folk,192


In [40]:
duplicates_per_playlist = sql_df("""
SELECT DISTINCT spt.playlist_name, COUNT(sd.n_duplicates) AS n_duplicates, spp.n_tracks
FROM spotify_playlists_tracks as spt
INNER JOIN spotify_duplicates AS sd
ON spt.id = sd.track_id
INNER JOIN songs_per_playlist as spp
ON spt.playlist_name = spp.playlist_name
GROUP BY 1, 3
ORDER BY spt.playlist_name ASC
""")
percentage_duplicates_per_playlist = sql_df("""
SELECT playlist_name, min(100*n_duplicates/n_tracks, 100) AS percentage_duplicates
FROM duplicates_per_playlist
ORDER BY percentage_duplicates DESC
""")
percentage_duplicates_per_playlist.to_csv("../data/percentage_duplicates_per_playlist.csv")
percentage_duplicates_per_playlist

,playlist_name,percentage_duplicates
0,covers mainstream (less rock),100
1,instrumental glory,100
2,snow,100
3,workout,97
4,comtemporary classical,96
...,...,...
65,dub,9
66,Menuo Juodaragis 2022,7
67,neofolk mainstream,5
68,salsa,3


In [41]:
duplicated_tracks_per_playlist = sql_df("""
SELECT DISTINCT spt.playlist_name, sd.artists_name, sd.track_name
FROM spotify_playlists_tracks as spt
INNER JOIN spotify_duplicates AS sd
ON spt.id = sd.track_id
ORDER BY 2, 3, 1
""")
duplicated_tracks_per_playlist.to_csv("../data/duplicated_tracks_per_playlist.csv")
duplicated_tracks_per_playlist

,playlist_name,artists_name,track_name
0,acoust&ballads&sg-sw/00s,"""Cats"" 1981 Original London Cast",Memory
1,amor,"""Cats"" 1981 Original London Cast",Memory
2,christmas,"""Cats"" 1981 Original London Cast",Memory
3,covers mainstream (less rock),"""Weird Al"" Yankovic",Angry White Boy Polka
4,just covers ( ͡° ͜ʖ ͡° ),"""Weird Al"" Yankovic",Angry White Boy Polka
...,...,...,...
3533,psytrance,zircon,The End
3534,acoust&ballads&sg-sw/00s,Édith Piaf,"Non, je ne regrette rien"
3535,glory with lyrics,Édith Piaf,"Non, je ne regrette rien"
3536,acoust&ballads&sg-sw 10s and beyond,Šventinis bankuchenas,Vasaraš Atostogaš
